In [1]:
import os
import json
import csv
import pandas as pd
import numpy as np
import math

In [2]:
results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_10_17_2023_merged.csv"

In [3]:
exp_df = pd.read_csv(results_csv_file)  

In [4]:
included_exp_names = exp_df['exp'].tolist()

In [5]:
def get_base_exp_name(exp_name):
    exp_field = "exp"
    exp_field_len_w_index = len(exp_field)+1
    exp_field_index = exp_name.find(exp_field)
    exp_name = exp_name[:exp_field_index]+exp_name[exp_field_index+exp_field_len_w_index:]
    return exp_name

In [6]:
exp_groups = dict()

for exp_name in included_exp_names:
    base_exp_name = get_base_exp_name(exp_name)
    if base_exp_name in exp_groups:
        exp_groups[base_exp_name] += [exp_name]
    else:
        exp_groups[base_exp_name] = [exp_name]

In [7]:
exp_length = 5
num_rounds = 5
round_strings =  ["round" + str(round_num) + "_test" for round_num in range(num_rounds)]
base_exp_names = list(exp_groups.keys())

In [18]:
results_dict = {}

for base_exp_name, exp_group in exp_groups.items():
    exp_dict = {}
    for i, run in enumerate(exp_group):
        sub_df = exp_df.loc[exp_df['exp'] == run]
        for round_string in round_strings:
            result = sub_df[round_string].iloc[0]
            if not math.isnan(result):
                if round_string not in exp_dict:
                    exp_dict[round_string] = [float(sub_df[round_string].item())]
                else:
                    exp_dict[round_string] += [float(sub_df[round_string].item())]
    results_dict[base_exp_name] = exp_dict
    print(base_exp_name)

DMPLS__coreset_cl_bs64_v5
DMPLS__coreset_cl_bs8_v5
DMPLS__coregcn_v5
DMPLS__coreset_cl_bs16_v5
DMPLS__coreset_fixed_v5
DMPLS__coreset_v5
DMPLS__coregcn_k_slice_wt_v5
DMPLS__coreset_umetric_ew1.0_v5
DMPLS__coreset_cl_bs240_v5
DMPLS__coregcn_inv_slice_wt_v5
DMPLS__coreset_umetric_ew5.0_v5
DMPLS__coregcn_exp_slice_wt_v5
DMPLS__coreset_cl_bs128_v5
DMPLS__coreset_cl_bs32_v5
DMPLS__coreset_cl_bs96_v5
DMPLS__coreset_umetric_ew10.0_v5


In [19]:
index = pd.Index(base_exp_names, name="exp_name")
round_counts_strings = [[round_string, f"round{i}_count"] for i, round_string in enumerate(round_strings)]
df = pd.DataFrame(index=index, columns=sum(round_counts_strings, []))

In [20]:
for base_exp_name, results in results_dict.items():
    sub_df = df.loc[base_exp_name]
    results = results_dict[base_exp_name]
    for round_string, count_string in round_counts_strings:
        if round_string in results:
            round_results = results[round_string]
            num_results = len(round_results)
            mean_result = np.mean(round_results)
            std_results = np.std(round_results, ddof=1)
            mean_string = '%.3f' % mean_result
            std_string = '%.3f'% std_results
            sub_df[round_string] = f"{mean_string}+/-{std_string}"
            sub_df[count_string] = num_results

/Users/arvin/.virtualenvs/al/lib/python3.9/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/arvin/.virtualenvs/al/lib/python3.9/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [21]:
df

,round0_test,round0_count,round1_test,round1_count,round2_test,round2_count,round3_test,round3_count,round4_test,round4_count
exp_name,,,,,,,,,,
DMPLS__coreset_cl_bs64_v5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DMPLS__coreset_cl_bs8_v5,0.730+/-0.001,2,0.786+/-0.004,2,0.834+/-0.002,2,0.839+/-0.003,2,0.860+/-0.001,2
DMPLS__coregcn_v5,0.702+/-0.054,5,0.772+/-0.022,5,0.825+/-0.007,5,0.842+/-0.009,5,0.854+/-0.015,5
DMPLS__coreset_cl_bs16_v5,0.686+/-0.028,3,0.793+/-0.004,2,0.818+/-0.019,2,0.838+/-0.004,2,0.862+/-0.007,2
DMPLS__coreset_fixed_v5,0.699+/-0.065,5,0.806+/-0.016,5,0.833+/-0.002,5,0.847+/-0.004,5,0.863+/-0.007,5
DMPLS__coreset_v5,0.665+/-0.114,5,0.793+/-0.013,5,0.837+/-0.005,5,0.848+/-0.009,5,0.863+/-0.010,5
DMPLS__coregcn_k_slice_wt_v5,0.729+/-0.036,5,0.774+/-0.021,5,0.815+/-0.003,5,0.819+/-0.008,5,0.856+/-0.006,5
DMPLS__coreset_umetric_ew1.0_v5,0.663+/-0.032,5,0.780+/-0.013,5,0.820+/-0.004,5,0.846+/-0.005,5,0.862+/-0.010,5
DMPLS__coreset_cl_bs240_v5,0.575+/-nan,1,0.768+/-nan,1,0.839+/-nan,1,0.847+/-nan,1,NaN,NaN
